# 🤖 Multilayer Perceptron

Esta Jupyter Notebook implementa una MLP para detectar patrones en una matriz 10x10.
Se genera un dataset para luego entrenar el modelo y validarlo.
Se evalúa la precisión para reiterar sobre el modelo con el objetivo de mejorar su eficacia.
Finalmente se exporta el modelo para ser utilizado en la aplicación web.


## 📋 Situación

En una matriz 10x10 se debe poder detectar las letras `b`, `d`, `f` que se corresponden a los siguientes patrones:

![Patrones que el MLP deberá reconocer](assets/patterns.png)

Ante un dato nuevo, el MLP deberá ser capaz de clasificar el contenido de esa matriz en uno de los tres patrones.


## 🗃️ Dataset

Se generan 3 datasets que contengan 100, 500 y 1000 ejemplos. 
El 10% serán patrones sin distorsionar y el resto con una distorsión del 1% al 30%.
Los datasets deberán ser representativos a la hora de definir la distribución de los ejemplos de entrenamiento.
Por cada dataset deberán construirse tres conjuntos de validación con 10%, 20% y 30% de los ejemplos.

En `dataset.py` se definen funciones de Python para ayudar a generar los datasets.


In [13]:
import numpy as np

# Define the 10x10 patterns for 'b', 'd', and 'f'
PATTERNS = {
    "b": np.array(
        [
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        ],
        dtype=np.uint8,
    ),
    "d": np.array(
        [
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        ],
        dtype=np.uint8,
    ),
    "f": np.array(
        [
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        ],
        dtype=np.uint8,
    ),
}


def generate_sample(pattern, noise=0.0):
    """
    Generates a 10x10 matrix based on a given pattern letter ('b', 'd', or 'f'), with optional noise.
    Args:
        pattern (str): One of 'b', 'd', or 'f'.
        noise (float): Proportion of pixels to flip.
    Returns:
        np.ndarray: A 10x10 numpy array with the pattern and noise applied.
    """
    assert pattern in PATTERNS, "Pattern must be one of 'b', 'd', or 'f'"
    assert 0.0 <= noise <= 0.30, "Noise must be between 0.0 and 0.30"

    matrix = PATTERNS[pattern].copy()
    num_pixels = matrix.size
    num_noisy = int(noise * num_pixels)

    if num_noisy > 0:
        # Choose random indices to flip
        indices = np.unravel_index(
            np.random.choice(num_pixels, num_noisy, replace=False), matrix.shape
        )
        # Flip the selected pixels (0->1, 1->0)
        matrix[indices] = 1 - matrix[indices]

    return matrix


print("A 'b' pattern with zero noise:")
print(generate_sample("d", 0.00))
print()
print("A 'b' with 30% noise, meaning 30 out of 100 cells have been randomly flipped:")
print(generate_sample("d", 0.30))

A 'b' pattern with zero noise:
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 1 1 1 1 0 0]
 [0 0 1 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 1 0 0]
 [0 0 0 1 1 1 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0]]

A 'b' with 30% noise, meaning 30 out of 100 cells have been randomly flipped:
[[1 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 1 0 0 0 0 0]
 [1 0 0 1 0 0 0 1 0 0]
 [0 0 1 1 0 1 1 1 0 0]
 [0 1 1 1 0 1 0 1 1 1]
 [1 0 1 0 0 0 0 1 1 0]
 [0 1 1 1 0 1 1 1 1 0]
 [1 1 1 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]]
